In [3]:
import requests
from bs4 import BeautifulSoup
import csv
import re

def extrair_filmes_imdb():
    url = "https://www.imdb.com/chart/top/"
    
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    }
    
    try:
        resposta = requests.get(url, headers=headers)
        resposta.raise_for_status()
        
        soup = BeautifulSoup(resposta.text, 'html.parser')
        
        filmes = []
        
        titulos = soup.find_all('h3', class_='ipc-title__text', limit=20)
        
        for titulo in titulos:
            filme_container = titulo.find_parent('div', class_='sc-5bc66c50-0')
            
            if filme_container:
                texto_titulo = titulo.text.strip()
                rank = re.search(r'^(\d+)\.', texto_titulo).group(1)
                nome = texto_titulo.split('. ', 1)[1]
                
                nota = filme_container.find('span', class_='ipc-rating-star--rating').text.strip()
                
                dados = filme_container.find_all('span', class_='sc-5bc66c50-6')
                
                ano = ""
                duracao = ""
                classificacao = ""
                
                for d in dados:
                    texto = d.text.strip()
                    if len(texto) == 4 and texto.isdigit():  # Ano
                        ano = texto
                    elif 'h' in texto or 'm' in texto:  # Duração
                        duracao = texto
                    elif len(texto) <= 2 and texto.isdigit():  # Classificação
                        classificacao = texto
                
                filmes.append({
                    'rank': rank,
                    'titulo': nome,
                    'nota': nota,
                    'ano': ano,
                    'duracao': duracao,
                    'classificacao': classificacao
                })
        
        with open('top20_filmes_imdb.csv', 'w', newline='', encoding='utf-8') as file:
            writer = csv.DictWriter(file, fieldnames=['rank', 'titulo', 'nota', 'ano', 'duracao', 'classificacao'])
            writer.writeheader()
            writer.writerows(filmes)
            
        print("Dados extraídos e salvos com sucesso.")
        return filmes
        
    except Exception as e:
        print(f"Ocorreu um erro: {str(e)}")
        return None

if __name__ == "__main__":
    extrair_filmes_imdb()

Dados extraídos e salvos com sucesso.
